# Settings

In [ ]:
CASE = 0
exceptions = []

match CASE:
    case 0: 
        IN_FILENAME = "../data/processed/filtered.xlsx"
        OUT_FILENAME = "../data/product/product.xlsx"
        
    case 1:
        IN_FILENAME = "../data/processed/filtered-nfv.xlsx"
        OUT_FILENAME = "../data/product/product-nfv.xlsx"
        
    case 2:
        IN_FILENAME = "../data/processed/filtered-ffu.xlsx"
        OUT_FILENAME = "../data/product/product-ffu.xlsx"
        exceptions.append("isFamiliar")

# adjustments

In [64]:
import re

fmt = re.search("[.][a-z]+$", OUT_FILENAME)
if fmt == None: raise Exception("have to choose a format (Xlsx | csv)")
fmt = fmt.group()[1:]

# Prevs

### Imports

In [65]:
import pandas as pd
import numpy as np
from collections import namedtuple
import json

In [66]:
with open("../constants/columns.json", "r") as file: columns, years = json.load(file).values()
columns = namedtuple("Columns", " ".join(columns.keys()))(**columns)
years = namedtuple("Years", re.sub(r'[\[\],\']', "", str([*map(lambda y: f'y{y}', years)])))(*years)

In [67]:
with open("../constants/filtered_columns.json", "r") as file:
    fcolumns = json.load(file)
    
for elm in exceptions: del fcolumns[elm]
    
fcolumns = namedtuple("FColumns", " ".join(fcolumns))(**fcolumns)

### tools

In [ ]:
YYYY = lambda txt, year : txt.replace('[YYYY]', str(year))

# Main

In [ ]:
data = pd.read_excel(IN_FILENAME)

In [ ]:
y_years = [2022]

df = pd.DataFrame(columns=[*fcolumns._asdict().keys()])

for row in data.iloc:
	df = pd.concat([
		df, pd.DataFrame([[row[YYYY(col, year if col != fcolumns.z_cat else year+2) ]  for col in fcolumns] for year in y_years], columns=[*fcolumns._asdict().keys()])
		], ignore_index=True)


In [ ]:
df.head()

# Save dataframe

In [ ]:
match fmt:
    case "xlsx": df.to_excel(OUT_FILENAME, index=False)
    case "csv": df.to_csv(OUT_FILENAME, index=False)
    case _: raise Exception("Stop touching things!!")